In [1]:
const name = "CombineImplicitSignals"
const residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

## Compute priors

## Load alphas

In [3]:
const alphas = [
    ["UniformBaseline"]
    ["PopularityBaseline"]
]

2-element Vector{String}:
 "UniformBaseline"
 "PopularityBaseline"

In [4]:
function get_indep(split, alphas)
    users = get_split(split).user
    X = zeros(Float32, length(users), length(alphas))
    @showprogress for j = 1:length(alphas)
        X[:, j] = get_alpha(alphas[j], split).rating
    end
    X
end;

In [5]:
const X = get_indep("validation", alphas);

In [6]:
const validation_weights = get_weights("validation", "inverse");

In [7]:
for j = 1:size(X)[2]
    @info sparse_crossentropy(X[:, j], validation_weights)
end

[ Info: 20220514 19:38:03 9.849667
[ Info: 20220514 19:38:04 7.40994


## Train a linear model

In [8]:
function loss(coefs)
    coefs = coefs .^ 2
    if sum(coefs) == 0
        coefs .+= 1 # prevent division by zero
    end
    coefs = coefs ./ sum(coefs)

    y = sum(X .* coefs', dims = 2)
    sparse_crossentropy(y, validation_weights)
end;

In [9]:
const res = optimize(
    loss,
    fill(1.0, size(X)[2]),
    LBFGS(manifold = Optim.Sphere()),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
)

Iter     Function value   Gradient norm 
     0     7.928072e+00     7.398828e-01
 * Current step size: 1.0
 * time: 0.9028129577636719
 * g(x): [0.7398827641024217, -0.7398827641024217]
 * x: [0.7071067811865475, 0.7071067811865475]
     1     7.410049e+00     1.670481e-02
 * Current step size: 0.9812959189663649
 * time: 4.358966827392578
 * g(x): [-0.016704814912505517, -0.00022073162132887065]
 * x: [-0.013212499214714665, 0.9999127111225765]
     2     7.409939e+00     1.858568e-05
 * Current step size: 0.82524021846523
 * time: 5.405544996261597
 * g(x): [1.858567834517883e-5, -2.749764016013462e-10]
 * x: [1.479506943272697e-5, 0.9999999998905529]
     3     7.409939e+00     2.008429e-14
 * Current step size: 1.0064275459508656
 * time: 6.967599868774414
 * g(x): [2.0084290346308516e-14, -3.2110833437343314e-28]
 * x: [1.5988034868877144e-14, 1.0]


 * Status: success

 * Candidate solution
    Final objective value:     7.409939e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.48e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.48e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.37e-10 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.86e-11 ≰ 0.0e+00
    |g(x)|                 = 2.01e-14 ≤ 1.0e-08

 * Work counters
    Seconds run:   7  (vs limit Inf)
    Iterations:    3
    f(x) calls:    8
    ∇f(x) calls:   9


In [10]:
res

 * Status: success

 * Candidate solution
    Final objective value:     7.409939e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.48e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.48e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.37e-10 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.86e-11 ≰ 0.0e+00
    |g(x)|                 = 2.01e-14 ≤ 1.0e-08

 * Work counters
    Seconds run:   7  (vs limit Inf)
    Iterations:    3
    f(x) calls:    8
    ∇f(x) calls:   9


# Evaluate on the test set

In [11]:
β = Optim.minimizer(res) .^ 2
β = β / sum(β)

2-element Vector{Float64}:
 2.556172589684314e-28
 1.0

In [12]:
sparse_crossentropy(get_indep("test", alphas) * β, get_weights("test", "inverse"))

7.409034945070174

In [13]:
# SOTA: 6.4687146803286275

In [14]:
write_params(Dict("β" => β, "alphas" => alphas));